In [2]:
import os
import csv
import torch
import logging
import numpy as np
import pandas as pd
# from transformers import *
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch.nn.functional as F

In [4]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification, AutoTokenizer, pipeline

# model_name = "hfl/chinese-xlnet-base"
model_name = "liam168/c2-roberta-base-finetuned-dianping-chinese"
# class_num = 4 # weibo-text.csv, 0:happy, 1:angry, 2:disgust, 3:down/upset
class_num = 2 # positive or negative
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=class_num)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
max_len = 200
device

Downloading: 100%|██████████| 854/854 [00:00<00:00, 227kB/s]
Downloading: 100%|██████████| 390M/390M [00:20<00:00, 19.5MB/s]
Downloading: 100%|██████████| 377/377 [00:00<00:00, 173kB/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 2.13MB/s]
Downloading: 100%|██████████| 263k/263k [00:00<00:00, 2.50MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 115kB/s]


device(type='cpu')

In [5]:
# model

In [7]:
df = pd.read_csv('./weibo-text.csv', encoding='utf-8')
df['text'].loc[0]

'啊呀呀 要死 穿 外套 认为 里面 件 余 周小伦 喜歡 各種 五角星 項鏈 露胸 衣服 伦伦 真的 越来越 帅'

In [11]:
ts_texts = ["我喜欢下雨。", "我讨厌他."]
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [12]:
classifier(ts_texts[0])

[{'label': 'positive', 'score': 0.9902541041374207}]

In [13]:
classifier(ts_texts[1])

[{'label': 'negative', 'score': 0.9941973686218262}]

In [21]:
classifier(df['text'].loc[0])

[{'label': 'positive', 'score': 0.9864313006401062}]

In [26]:
df['text'].loc[:3].values.astype(str)

array(['啊呀呀 要死 穿 外套 认为 里面 件 余 周小伦 喜歡 各種 五角星 項鏈 露胸 衣服 伦伦 真的 越来越 帅',
       '风格 喜欢 喜欢 张',
       '试试 去死皮 面膜 燕麦片 加水 中 浸泡 小时 加入 木瓜 牛奶 搅拌 放入 压缩 纸 面膜 敷于 脸上 分钟 清水 洗净 改善 粗糙 肌肤 去除 死皮 肌肤 光滑',
       '张老师 谢谢 侬 信任 粉丝 无所谓 重 质地 近日 发现 现象 加 关注 回加 立即 取消 关注 回应 立即 取消 关注'],
      dtype='<U83')

In [28]:
test_len = 100
model_output = []
for i in range(test_len):
  model_output.append(classifier(df['text'].loc[i])[0])

In [30]:
# model_output

In [32]:
new_df = pd.read_csv("chnsenticorp/dev.tsv", sep="\t")

In [33]:
new_df.head()

,label,text_a
0,1,"這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~..."
1,1,"<荐书> 推荐所有喜欢<红楼>的红迷们一定要收藏这本书,要知道当年我听说这本书的时候花很长时..."
2,0,商品的不足暂时还没发现，京东的订单处理速度实在.......周二就打包完成，周五才发货...
3,1,２００１年来福州就住在这里，这次感觉房间就了点，温泉水还是有的．总的来说很满意．早餐简单了些．
4,1,不错的上网本，外形很漂亮，操作系统应该是个很大的 卖点，电池还可以。整体上讲，作为一个上网本...


In [55]:
model_output.shape

(200,)

In [ ]:
test_len = 200
model_output = []
for i in range(test_len):
  one_output = classifier(new_df['text_a'].loc[i])[0]
  one_output = 1 if one_output['label'] == 'positive' else 0
  model_output.append(one_output)
model_output = np.array(model_output)

In [56]:
true_labels = new_df['label'].loc[:199].values
true_labels.shape

(200,)

In [58]:
model_output.shape

(200,)

In [59]:
np.sum(model_output==true_labels)

173

In [8]:
train_encodings = tokenizer(df['text'].loc[0], return_tensors='pt', padding=True)
train_encodings['input_ids']

tensor([[ 101, 1557, 1435, 1435, 6206, 3647, 4959, 1912, 1947, 6371,  711, 7027,
         7481,  816,  865, 1453, 2207,  840, 1599, 3631, 1392, 4934,  758, 6235,
         3215, 7517, 7122, 7463, 5541, 6132, 3302,  840,  840, 4696, 4638, 6632,
         3341, 6632, 2358,  102]])

In [9]:
encoded_dict = tokenizer.encode_plus(
            df['text'].loc[0],
            add_special_tokens = True,
            max_length = 200,
            padding='longest',
            return_tensors = 'pt',
        )
model(encoded_dict['input_ids'], token_type_ids=encoded_dict['token_type_ids'], attention_mask=encoded_dict['attention_mask'], labels=torch.tensor([1]))

SequenceClassifierOutput(loss=tensor(0.0137, grad_fn=<NllLossBackward0>), logits=tensor([[-2.1138,  2.1725]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [7]:
# def process_data(filename):
#   with open(filename, 'r') as f:
#     rows = [row for row in csv.reader(f)]
#     rows = np.array(rows[1:])
#     comments = []
#     labels = []
#     for c, l in rows:
#       comments.append(c)
#       labels.append(l)
#     classes = list(set(labels))
#     num_classes = len(classes)
#     # print(comments, labels)
#     return  comments,  labels, classes,   num_classes
def process_data(filename="text.csv"):
    f = pd.read_csv(filename)
    comment = f['text'].to_numpy()
    label = f['label'].to_numpy()
    return comment, label, set(label)

# a, b, c = process_data()

In [40]:
def encode(comments, labels, max_len):
    input_id_list, token_type_ids, attention_mask = [], [], []
    print("len78")
    print(len(comments))
    # print(type(input_id_list))
    for i in range(len(comments)):
        encoded_dict = tokenizer.encode_plus(
                comments[i],
                add_special_tokens = True,
                max_length = max_len,
                pad_to_max_length = True,
                return_tensors = 'pt',
            )

        # print(type(input_id_list))
        input_id_list.append(encoded_dict['input_ids'])
        token_type_ids.append(encoded_dict['token_type_ids'])
        attention_mask.append(encoded_dict['attention_mask'])

    # onehot_encoder = OneHotEncoder()
    # labels = onehot_encoder.fit_transform(np.array(labels).reshape(-1, 1)).toarray()

    input_id_list = torch.cat(input_id_list, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    attention_mask = torch.cat(attention_mask, dim=0)

    input_id_list = torch.LongTensor(input_id_list)
    token_type_ids = torch.LongTensor(token_type_ids)
    attention_mask = torch.LongTensor(attention_mask)
    labels = torch.LongTensor(labels)
    print("len103")
    print(len(input_id_list))
    return input_id_list, token_type_ids, attention_mask, labels

In [57]:
import torch.nn.functional

output_dir = './models/'
output_model_file = os.path.join(output_dir, 'parameters')
output_config_file = os.path.join(output_dir, 'config')

def save(model):
    # save
    torch.save(model.state_dict(), output_model_file)
    model.config.to_json_file(output_config_file)

def eval(model, dataloader):
    model.eval()
    loss, num_examples, accuracy, correct_pred = 0, 0, 0, 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            logits = model(batch[0], token_type_ids=batch[1], attention_mask=batch[2])[0]
            probas = torch.nn.functional.softmax(logits, dim = 1)
            logits = logits.detach().cpu().numpy()
            label = batch[3]
            probas = torch.nn.functional.softmax(logits, label)

            loss += torch.nn.functional.cross_entropy(logits, label).item()

            _, predicted_labels = torch.max(probas, 1)
            num_examples += label.size(0)
            correct_pred += (predicted_labels == label).sum()
            # eval_accuracy += tmp_eval_accuracy
            # nb_eval_steps += 1
    return correct_pred.item()/num_examples * 100, loss

In [58]:
eval(model, train_dataloader)

AttributeError: 'numpy.ndarray' object has no attribute 'softmax'

In [42]:
from transformers import AutoModelForCausalLM, AdamW

batch_size = 2

comments,  labels, classes = process_data()
num_classes = len(classes)

# input_ids, token_type_ids, attention_mask, labels = encode(comments,  labels, max_len)
input_ids, token_type_ids, attention_mask, labels = encode(comments,  labels, max_len)

len78
265433


/Users/kesong/miniconda3/envs/exercism/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


len103
265433


In [43]:
input_train, input_test, label_train, label_test_ = train_test_split(input_ids, labels, test_size=0.4, random_state=42)
input_val, input_test, label_val, label_test = train_test_split(input_test, label_test_, test_size=0.5, random_state=42)

token_train, token_test, _, _ = train_test_split(token_type_ids, labels, test_size=0.4, random_state=42)
token_val, token_test,  _, _ = train_test_split(token_test, label_test_, test_size=0.5, random_state=42)

mask_train, mask_test, _, _ = train_test_split(attention_mask, labels, test_size=0.4, random_state=42)
mask_val, mask_test,  _, _ = train_test_split(mask_test, label_test_, test_size=0.5, random_state=42)

train_data = Data.TensorDataset(input_train, token_train, mask_train, label_train)
train_dataloader = Data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_data = Data.TensorDataset(input_val, token_val, mask_val, label_val)
val_dataloader = Data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

test_data = Data.TensorDataset(input_test, token_test, mask_test, label_test)
test_dataloader = Data.DataLoader(test_data, batch_size=batch_size, shuffle=True)


In [44]:
for i, batch in enumerate(train_dataloader):
  print("i=", i)
  batch = tuple(t.to(device) for t in batch)
  break

i= 0


In [45]:
batch[3].shape

torch.Size([2])

In [46]:
loss = model(batch[0], token_type_ids=batch[1], attention_mask=batch[2])

In [37]:
loss.logits

tensor([[-0.1730,  0.0845,  0.0520,  0.3221],
        [-0.1082,  0.2527, -0.1241,  0.3594]], grad_fn=<AddmmBackward0>)

In [49]:
loss = model(batch[0], token_type_ids=batch[1], attention_mask=batch[2], labels=batch[3])

In [60]:

def train(model_name, epochs):
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}]

  optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

  for _ in range(epochs):
        for i, batch in enumerate(train_dataloader):
            print("i=", i)
            batch = tuple(t.to(device) for t in batch)
            loss = model(batch[0], token_type_ids=batch[1], attention_mask=batch[2], labels=batch[3])[0]
            # predictions = F.softmax(logits, dim=-1)

            print(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # if i % 10 == 0:
            #   train_acc, train_loss = eval(model, train_dataloader)
            #   val_acc, val_loss = eval(model, val_dataloader)

            #   print(f'Train Accuracy: {train_acc:.2f}% Loss: {train_loss:.4f}, Val Accuracy: {val_acc:.2f}% ')

train(model_name, 2)

i= 0
1.3852424621582031
i= 1
1.3356711864471436
i= 2
1.186072826385498
i= 3
1.2101402282714844
i= 4
1.5175080299377441
i= 5
1.3342182636260986
i= 6
1.0642211437225342
i= 7
1.2318025827407837
i= 8
1.0291621685028076
i= 9
0.8866887092590332
i= 10
0.7873426079750061
i= 11
0.7226966619491577
i= 12
0.601564347743988
i= 13
1.8192312717437744


In [43]:
# len(comments)
# len(labels)
# input_train, input_test, label_train, label_test = train_test_split(input_ids, labels, test_size=0.4, random_state=42)
# input_ids, token_type_ids, attention_mask, labels = encode(comments,  labels, max_len)
# print(input_ids.shape)
# print(labels.shape)
print(train_data.shape)
# print(len(input_test))
# print(len(label_train))
# print(len(label_test ))

NameError: name 'train_data' is not defined